<a href="https://colab.research.google.com/github/lmillana/MasterProjects/blob/main/Machine%20Learning/%20Time%20Series%20Classification%20with%20RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación de series temporale desde cero

Entrenamiento de un clasificador de series temporales desde cero en el conjunto de datos FordA del archivo UCR/UEA.

## Introducción

Este ejemplo muestra cómo realizar la clasificación de series temporales desde cero, partiendo de archivos de series temporales CSV sin procesar en disco.
CSV en disco.

Demostramos el flujo de trabajo con el conjunto de datos FordA de la base de datos
[UCR/UEA archivo](https://www.cs.ucr.edu/%7Eeamonn/time_series_data_2018/).

## Setup

In [ ]:
!pip install tensorflow

In [ ]:
# instala las librerías necesarias para el proyecto

import numpy as np

from tensorflow import keras
from tensorflow.python.keras import layers

from keras.models import Sequential
from keras.layers import Embedding, LSTM, GRU, SimpleRNN,  Dense, Dropout



## Cargar los datos: el dataset FordA

### Descripción del dataset

El conjunto de datos que utilizamos aquí se llama FordA.
Los datos proceden del archivo de la UCR.
El conjunto de datos contiene 3.601 instancias de entrenamiento y otras 1.320 instancias de prueba.
Cada serie temporal corresponde a una medición del ruido del motor captada por un sensor del motor.
Para esta tarea, el objetivo es detectar automáticamente la presencia de un problema específico en el motor.


El problema es una **tarea de clasificación binaria equilibrada**. La descripción completa de este conjunto de datos [aquí](http://www.j-wichard.de/publications/FordPaper.pdf).

### Leer los datos TSV

Utilizaremos el archivo `FordA_TRAIN` para el entrenamiento y el archivo para las pruebas. La simplicidad de este conjunto de datos
nos permite demostrar eficazmente cómo utilizar RNN y CNN para la clasificación de series temporales.
En este archivo, la primera columna corresponde a la etiqueta.

In [ ]:
## OPCION 1: dividimos entre df_train & df_test
# habría que dividir después x_train, y_train & x_test, y_test

import pandas as pd

root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

df_train = pd.read_csv(root_url + "FordA_TRAIN.tsv", sep= "\t", header = None)
df_test = pd.read_csv(root_url + "FordA_TEST.tsv", sep= "\t", header = None)

In [ ]:
df_test.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,500
0,-1,-0.140402,0.171641,0.302044,0.232804,0.033853,-0.224183,-0.469987,-0.645396,-0.617700,...,-0.319966,0.390903,0.974831,1.258717,1.143316,0.647092,-0.049582,-0.690402,-0.976596,-0.794263
1,-1,0.334038,0.322253,0.453844,0.671852,0.887897,1.020469,1.059750,1.030290,0.950746,...,0.435186,-0.346502,-0.924912,-1.208716,-1.247996,-1.139974,-1.041772,-1.041772,-1.159614,-1.375659
2,-1,0.716686,0.744367,0.725913,0.661325,0.555217,0.413585,0.246580,0.065273,-0.121109,...,3.171020,2.276019,1.219548,0.081881,-1.050250,-2.092881,-2.983269,-3.675281,-4.136622,-4.339612
3,1,1.240282,1.331189,1.386596,1.383220,1.305979,1.142784,0.878613,0.532291,0.140025,...,-0.820262,-1.124551,-1.302012,-1.340564,-1.271440,-1.146352,-1.011328,-0.931222,-0.934498,-1.001288
4,-1,-1.159478,-1.204174,-1.167605,-1.033518,-0.818166,-0.558119,-0.299291,-0.093691,0.022770,...,0.660853,0.441438,0.206176,-0.006941,-0.146919,-0.183082,-0.112382,0.008987,0.131413,0.186266


In [ ]:
## OPCION 2: dividimos x_train, y_train & x_test, y_test

# lee un TSV
# utiliza numpy o pandas para leerlo
def readucr (filename):
  data = np.loadtxt(filename, delimiter = "\t")
  y = data[:, 0]
  x = data [:, 1:]
  return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

# lee un TSV
# utiliza numpy o pandas para leerlo

# obtén x_train, x_test, y_train, y_test
x_train, y_train = readucr (root_url + "FordA_TRAIN.tsv") #500
x_test, y_test = readucr (root_url + "FordA_TEST.tsv") #500


In [ ]:
## LONGITUD
print(x_train.shape) #(3601, 500)
print(y_train.shape) #(3601,)
print(x_test.shape) #(1320, 500)
print(y_test.shape) #(1320,)

(3601, 500)
(3601,)
(1320, 500)
(1320,)


In [ ]:
## Comprobamos que los datos de entrenamiento están BALANCEADOS

val, frec = np.unique(y_train, return_counts=True)

print(val, frec)

[-1  1] [1846 1755]


## Visualizar los datos

Aquí visualizamos un ejemplo de serie temporal para cada clase del conjunto de datos.

In [ ]:
# tienes que tener en cuenta las clases que hay y visualizar las series de cada clase
# no tine que ser igual el gráfico

import matplotlib.pyplot as plt

class1 = df_train[df_train[0] == 1]
class_1 = df_train[df_train[0] == -1]



## Estandarizar los datos

Nuestras series temporales ya tienen una única longitud (500). Sin embargo, sus valores
suelen estar en varios rangos. Esto no es lo ideal para una red neuronal;
en general, deberíamos intentar normalizar los valores de entrada.

Para este conjunto de datos concreto, los datos ya están normalizados en z: cada muestra de serie temporal
tiene una media igual a cero y una desviación estándar igual a uno.

Este tipo de normalización es muy común en los problemas de clasificación de series temporales.
[Bagnall et al. (2016)](https://link.springer.com/article/10.1007/s10618-016-0483-9).

Tenga en cuenta que los datos de series temporales utilizados aquí son univariantes, lo que significa que sólo tenemos un canal
por ejemplo de serie temporal.
Por lo tanto, transformaremos la serie temporal en una multivariante con un canal
utilizando una simple remodelación a través de numpy.
Esto nos permitirá construir un modelo fácilmente aplicable a series temporales multivariantes.
multivariantes.

In [ ]:
# Utiliza el reshape en x_test y x_train
# x_train tiene que ser de la forma (x_train.shape[0],x_train.shape[1],1)

## reshape((array que cambiamos, new shape, num canal))
## old: (3601, 500)
x_train = x_train.reshape((x_train.shape[0],x_train.shape[1],1) )

# x_test tiene que ser de la forma (x_test.shape[0],x_test.shape[1],1)
## old: (1320, 500)
x_test = x_test.reshape((x_test.shape[0],x_test.shape[1],1))



In [ ]:
print('new shape x_train: ', x_train.shape)
print('new shape x_test: ', x_test.shape)

new shape x_train:  (3601, 500, 1)
new shape x_test:  (1320, 500, 1)


Por último, para utilizar `sparse_categorical_crossentropy`, tendremos que contar el número de clases de antemano.

In [ ]:
num_classes = np.unique(y_train) # cuenta el numero de clases

print(f'Tenemos {len(num_classes)} clases: {num_classes}')

Tenemos 2 clases: [-1  1]


Ahora barajamos el conjunto de entrenamiento porque vamos a utilizar la opción `validation_split`
más tarde cuando entrenemos.

In [ ]:
# para barajar los datos, puedes utilizar np.random.permutation(longitud datos train)
idx = np.random.permutation(len(x_train)) # utiliza lo anterior para generar un índice aleatorio

# separa cada train con ese índice
x_train = x_train[idx]
y_train = y_train[idx]

x_train.shape, y_train.shape

((3601, 500, 1), (3601,))

In [ ]:
y_train

array([ 1,  1, -1, ...,  1, -1, -1])

Se normalizan las etiquetas a números enteros positivos.
Las etiquetas esperadas serán entonces 0 y 1.

In [ ]:
# para normalizar, tienes que quedarte con los valores de y_train == -1
# estos valores van a ser ahora 0

y_train = np.where(y_train == -1, 0, y_train)
y_test = np.where(y_test == -1,0, y_test)

y_train


array([0, 0, 0, ..., 1, 1, 1])

## Construyendo el modelo

En este punto, se pueden crear muchos tipos.

Para series temporales puedes utilizar las siguientes:
- SimpleRNN
- LSTM
- GRU
- Conv1D

Y algunas más que no vamos a hablar aquí.

Intenta probar con RNN, LSTM y GRU por sencillez.

In [ ]:
x_train.shape[1]

500

In [ ]:
#from keras.models import Sequential
#from keras.layers import Embedding, LSTM, GRU, SimpleRNN,  Dense

# @title PRIMER MODELO: RNN SIMPLE

input_shape = (x_train.shape[1], 1) #datos de entrada

RNN = Sequential()

RNN.add(SimpleRNN(32, input_shape=input_shape)) # Capa RNN, 64 vectores
RNN.add(Dropout(0.5)) #Prevenimos sobreajuste
RNN.add(Dense(1, activation='sigmoid'))


In [ ]:
# @title ENTRENAMIENTO

## CREAMOS MODELO:
# compila el modelo eligiendo optimizer, loss y metrics
RNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])


In [ ]:
## ENTRENAMOS:

# Define las épocas y batch_size
epochs = 200 #muchos datos por lo que se necesitan muchas epochs
batch_size = 64

# entrena el modelo y almacena el entrenamiento en la variable history
history = RNN.fit(x_train, y_train,
                  epochs= epochs,
                  batch_size=batch_size,
                  validation_split= 0.2)


# guarda el modelo en .h5
RNN.save('RNN.h5')

Epoch 1/200
45/45 [==============================] - 9s 129ms/step - loss: 0.7133 - acc: 0.5170 - val_loss: 0.6931 - val_acc: 0.5035
Epoch 2/200
45/45 [==============================] - 5s 103ms/step - loss: 0.7090 - acc: 0.5010 - val_loss: 0.6927 - val_acc: 0.5090
Epoch 3/200
45/45 [==============================] - 6s 126ms/step - loss: 0.7022 - acc: 0.5212 - val_loss: 0.6919 - val_acc: 0.5284
Epoch 4/200
45/45 [==============================] - 4s 100ms/step - loss: 0.6972 - acc: 0.5097 - val_loss: 0.6920 - val_acc: 0.5312
Epoch 5/200
45/45 [==============================] - 6s 131ms/step - loss: 0.7047 - acc: 0.5073 - val_loss: 0.7004 - val_acc: 0.5035
Epoch 6/200
45/45 [==============================] - 4s 98ms/step - loss: 0.7067 - acc: 0.4986 - val_loss: 0.6919 - val_acc: 0.5243
Epoch 7/200
45/45 [==============================] - 5s 106ms/step - loss: 0.7048 - acc: 0.5052 - val_loss: 0.6924 - val_acc: 0.5326
Epoch 8/200
45/45 [==============================] - 6s 123ms/step - l


## Evaluar el modelo en los datos de test

In [ ]:
from keras.models import load_model

# carga el modelo
model = load_model('RNN.h5')

test_loss, test_acc = model.evaluate(x_test, y_test)

# evalua el modelo con evaluate y saca test_loss y test_acc

print("Test accuracy", test_acc)
print("Test loss", test_loss)

42/42 [==============================] - 2s 36ms/step - loss: 0.6928 - acc: 0.5182
Test accuracy 0.5181818008422852
Test loss 0.6928234696388245


In [ ]:

# @title SEGUNDO MODELO: LSTM

model_LSTM = Sequential()

model_LSTM.add(LSTM(64, return_sequences=True, input_shape=input_shape)) # Capa RNN, 32 vectores
model_LSTM.add(Dropout(0.5))  # Capa Dropout para prevenir sobreajuste
model_LSTM.add(LSTM(32)) # Capa RNN, 32 vectores
model_LSTM.add(Dropout(0.5))
model_LSTM.add(Dense(1, activation='sigmoid'))

In [ ]:
# @title ENTRENAMIENTO

## CREAMOS MODELO:
# compila el modelo eligiendo optimizer, loss y metrics
model_LSTM.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
## ENTRENAMOS:

# Define las épocas y batch_size
epochs = 200
batch_size = 64

# entrena el modelo y almacena el entrenamiento en la variable history
history = model_LSTM.fit(x_train, y_train,
                  epochs= epochs,
                  batch_size=batch_size,
                  validation_split=0.2)


# guarda el modelo en .h5
model_LSTM.save('LSTM.h5')

Epoch 1/200
45/45 [==============================] - 5s 53ms/step - loss: 0.6926 - acc: 0.5260 - val_loss: 0.6923 - val_acc: 0.5215
Epoch 2/200
45/45 [==============================] - 1s 31ms/step - loss: 0.6934 - acc: 0.5021 - val_loss: 0.6918 - val_acc: 0.5243
Epoch 3/200
45/45 [==============================] - 1s 32ms/step - loss: 0.6933 - acc: 0.5083 - val_loss: 0.6923 - val_acc: 0.5201
Epoch 4/200
45/45 [==============================] - 2s 45ms/step - loss: 0.6949 - acc: 0.5052 - val_loss: 0.6923 - val_acc: 0.5201
Epoch 5/200
45/45 [==============================] - 2s 38ms/step - loss: 0.6930 - acc: 0.5260 - val_loss: 0.6932 - val_acc: 0.5187
Epoch 6/200
45/45 [==============================] - 1s 31ms/step - loss: 0.6936 - acc: 0.5163 - val_loss: 0.6925 - val_acc: 0.5243
Epoch 7/200
45/45 [==============================] - 2s 35ms/step - loss: 0.6929 - acc: 0.5267 - val_loss: 0.6911 - val_acc: 0.5298
Epoch 8/200
45/45 [==============================] - 1s 32ms/step - loss: 0.

In [ ]:
## EVALUAMOS:

# carga el modelo
model = load_model('LSTM.h5')

test_loss, test_acc = model.evaluate(x_test, y_test)

# evalua el modelo con evaluate y saca test_loss y test_acc

print("Test accuracy", test_acc)
print("Test loss", test_loss)

42/42 [==============================] - 1s 16ms/step - loss: 0.6927 - acc: 0.5159
Test accuracy 0.5159090757369995
Test loss 0.6926639676094055


In [ ]:

# @title TERCER MODELO: GRU

model_GRU = Sequential()

model_GRU.add(GRU(64)) # Capa RNN, 32 vectores
model_GRU.add(Dropout(0.5))
model_GRU.add(Dense(1, activation='sigmoid'))

In [ ]:
# @title ENTRENAMIENTO

## CREAMOS MODELO:
# compila el modelo eligiendo optimizer, loss y metrics
model_GRU.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
## ENTRENAMOS:

# Define las épocas y batch_size
epochs = 200
batch_size = 64

# entrena el modelo y almacena el entrenamiento en la variable history
history = model_GRU.fit(x_train, y_train,
                  epochs= epochs,
                  batch_size=batch_size,
                  validation_split=0.2)


# guarda el modelo en .h5
model_GRU.save('GRU.h5')

Epoch 1/200
45/45 [==============================] - 4s 41ms/step - loss: 0.6957 - acc: 0.4983 - val_loss: 0.6923 - val_acc: 0.5326
Epoch 2/200
45/45 [==============================] - 1s 26ms/step - loss: 0.6947 - acc: 0.5017 - val_loss: 0.6927 - val_acc: 0.5132
Epoch 3/200
45/45 [==============================] - 1s 20ms/step - loss: 0.6948 - acc: 0.5122 - val_loss: 0.6923 - val_acc: 0.5187
Epoch 4/200
45/45 [==============================] - 1s 18ms/step - loss: 0.6953 - acc: 0.5031 - val_loss: 0.6916 - val_acc: 0.5215
Epoch 5/200
45/45 [==============================] - 1s 17ms/step - loss: 0.6928 - acc: 0.5146 - val_loss: 0.6919 - val_acc: 0.5298
Epoch 6/200
45/45 [==============================] - 1s 17ms/step - loss: 0.6929 - acc: 0.5201 - val_loss: 0.6929 - val_acc: 0.5118
Epoch 7/200
45/45 [==============================] - 1s 19ms/step - loss: 0.6946 - acc: 0.5184 - val_loss: 0.6917 - val_acc: 0.5173
Epoch 8/200
45/45 [==============================] - 1s 17ms/step - loss: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# carga el modelo
model = load_model('GRU.h5')

test_loss, test_acc = model.evaluate(x_test, y_test)

# evalua el modelo con evaluate y saca test_loss y test_acc

print("Test accuracy", test_acc)
print("Test loss", test_loss)

42/42 [==============================] - 1s 11ms/step - loss: 0.1657 - acc: 0.9394
Test accuracy 0.939393937587738
Test loss 0.16572408378124237


# Conclusión

Qué puedes decir de estos modelos?

Hay sobreajuste?

Entrenan bien?



---



**CONCLUSIONES**

* Se han probado varias combinaciones con **RNN** (32/64 capas, con/sin dropout, batch_size 32/64) sin llegar a superar el 0.51 de accuracy y una pérdida de 0.69.
* En RNN vemos sobreajuste y con cambios insignificante a pesar de las capas de Dropout.

* Con LSTM también se han probado varias combinaciones, incluso añadiendo dos capas LSTM (64 y 32)  y el máximo accuracy ha sido de 0.51 pero con mucha pérdida.

* El resultado con GRU es sorprendente. Con una sola capa GRU, se obtiene un resultado de rendimiento de 0.5. Sin embargo, si añadimos una capa de Dropout, **conseguimos un 0.939 con una pérdida de 0.165**, demostrando así su buen rendimiento.


**Nota**: Para optimizar las redes neuronales, se utiliza una herramienta llamada KerasTuner.
[Puedes echarle un vistazo si quieres en este enlace](https://keras.io/keras_tuner/)

In [ ]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.4 MB/s eta 0:00:00


In [ ]:
import keras_tuner

def build_model(hp):
  model_tuner = Sequential()
  model_tuner.add(Dense(
      hp.Choice('units', [8, 16, 32]),
      activation='relu'))
  model_tuner.add(Dense(1, activation='relu'))
  model_tuner.compile(loss='mse')
  return model_tuner


tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5)

# Start the search and get the best model
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))
best_model = tuner.get_best_models()[0]

print(best_model)

Trial 3 Complete [00h 00m 06s]
val_loss: 0.2504463195800781

Best val_loss So Far: 0.2499276101589203
Total elapsed time: 00h 00m 20s
